# Getting started with operators

Qiskit provides sophisticated functionality with operators: [The "Operator Flow" layer in Aqua](https://qiskit.org/documentation/apidoc/qiskit.aqua.operators.html). In this tutorial notebook we will get familiar with the main building blocks of the operator flow layer:

1. State functions
2. Operators and evolutions
3. Measurements and expectations

This tutorial assumes basic familiarity with qubits, quantum superposition and quantum circuits.

There are a few important underlying mathematical concepts:

* An n-qubit State function is a complex function over n binary variables, which we will often refer to as n-qubit binary strings. For example, the traditional quantum “zero state” is a 1-qubit state function, with a definition of f(0) = 1 and f(1) = 0.
* An n-qubit Operator is a linear function taking n-qubit state functions to n-qubit state functions. For example, the Pauli X Operator is defined by f(Zero) = One and f(One) = Zero. Equivalently, an Operator can be defined as a complex function over two n-qubit binary strings, and it is sometimes convenient to picture things this way. By this definition, our Pauli X can be defined by its typical matrix elements, f(0, 0) = 0, f(1, 0) = 1, f(0, 1) = 1, f(1, 1) = 0.
* An n-qubit Measurement is a functional taking n-qubit State functions to complex values. For example, a Pauli Z Measurement can be defined by f(Zero) = 0 and f(One) = 1.

The definitions of these concepts mostly follows the ones in the first two chapters of the book ["The Theory of Quantum Information" by John Watrous](https://cs.uwaterloo.ca/~watrous/TQI/), but don't worry if you don't get them at the first glance: This tutorial is all about getting intuition behind them and the structure of the operator flow layer.

First, let us import the necessary components:

In [1]:
from qiskit import BasicAer, QuantumCircuit
from qiskit.aqua.operators import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import Pauli
import numpy as np

## State functions

Let us start with a single qubit. As we know, a pure state of a qubit is, in general, described as a superposition of basis states $\vert \psi \rangle = \alpha \vert 0\rangle + \beta \vert 1 \rangle$ with complex coefficients $\alpha$ and $\beta$. We can think of this superposition as some mathematical object which puts into correspondence the ket $\vert 0\rangle$ with $\alpha$ and $\vert 1\rangle$ with $\beta$. This is exactly what the state functions in Aqua are. For example the well known minus state $\vert \psi_{-}\rangle=\frac{1}{\sqrt{2}}\vert 0 \rangle - \frac{1}{\sqrt{2}}\vert 1\rangle$ can be thought of as a function mapping the basis ket $\vert 0\rangle$ to $\frac{1}{\sqrt{2}}$ and the basis ket $\vert 1\rangle$ to $-\frac{1}{\sqrt{2}}$. Basis states themselves are particular cases of state functions, e.g. the basis state $\vert 0\rangle$ can be thought of as a function mapping the ket $\vert 0\rangle$ to the number $1$ and the ket $\vert 1\rangle$ to the number $0$.

In Aqua, the class `StateFn` can be used to create state functions. Let us create the 1-qubit state function corresponding to the state $\vert \psi_{-}\rangle$

In [2]:
amplitudes = [1, -1]
normalization_coefficient = 1 / np.sqrt(2)
psi_minus = StateFn(amplitudes, coeff=normalization_coefficient)

To access the mapping represented by this state function we can use the function `.eval()`

In [3]:
print(f"The ket |0> is mapped to {psi_minus.eval('0')}")
print(f"The ket |1> is mapped to {psi_minus.eval('1')}")

The ket |0> is mapped to (0.7071067811865475+0j)
The ket |1> is mapped to (-0.7071067811865475+0j)


As you can see we have explicitly specified the normalization coefficient to keep the state normalized. Essentially the Aqua state functions can be thought of and treated as ket vectors, but keep in mind that they are a bit more general mathematical objects and do not impose normalization constraint. At the end of the day, when you create quantum circuits out of state functions and sample them the normalization will be taken care of internally, so you actually don't need to explicitly specify the coefficient every time you create a state function (the default value is `coeff=1`).

If we look at the type of the object `psi_minus`

In [4]:
type(psi_minus)

qiskit.aqua.operators.state_fns.vector_state_fn.VectorStateFn

we can see that it is `VectorStateFn`. This is not much of a surprise, since we built the state function based on a vector of amplitudes. In how many ways can we create a state function? One way is obviously with a vector of complex amplitudes as done above. Another way is by specifying the mapping explicitly with a python dictionary

In [5]:
mapping = {'0': 1, '1': -1}
psi_minus_dict = StateFn(mapping, coeff=normalization_coefficient)
type(psi_minus_dict)

qiskit.aqua.operators.state_fns.dict_state_fn.DictStateFn

`DictStateFn` objects provide an additional convenience where you can omit the zero amplitudes, e.g. the state $\vert 1\rangle$ can be initialized with `{'1': 1}` instead of `{'0': 0, '1': 1}`.

Yet another way to build a state function object is using a quantum circuit

In [6]:
qc = QuantumCircuit(1)
qc.h(0)
qc.z(0)
psi_minus_qc = StateFn(qc)
type(psi_minus_qc)

qiskit.aqua.operators.state_fns.circuit_state_fn.CircuitStateFn

An object used to build a state function is called its _primitive_. The class `StateFn` is a convenient way to create state function objects with various primitives, however the underlying classes `VectorStateFn`, `DictStateFn`, `CircuitStateFn` etc. can be used directly as well, e.g. `DictStateFn({'0': 1, '1': -1})`. The full list of allowed primitives can be studied in the documentation. One important type of primitve is an Aqua operator, which we will cover briefly after we introduce operators and measurements.

To make the usage of state functions easier, Aqua provides the following frequently used 1-qubit state functions as pre-built objects

* $\vert 0 \rangle \rightarrow$ `Zero`
* $\vert 1 \rangle \rightarrow$ `One`
* $\frac{1}{\sqrt{2}} (\vert 0 \rangle + \vert 1 \rangle) \rightarrow$ `Plus`
* $\frac{1}{\sqrt{2}} (\vert 0 \rangle - \vert 1 \rangle) \rightarrow$ `Minus`

E.g. if you need to use the minus state in your code, you can use `Minus`:

In [7]:
print(f"'Minus' maps |0> to {Minus.eval('0')}")
print(f"'Minus' maps |1> to {Minus.eval('1')}")

'Minus' maps |0> to (0.7071067811865476-8.7e-17j)
'Minus' maps |1> to (-0.7071067811865476+8.7e-17j)


Multi-qubit state functions can be created in a manner identical to one-qubit state functions. As an example let us create a state function object for the 2-qubit Bell state $\vert \Phi^{+} \rangle = \frac{1}{\sqrt{2}}(\vert 00 \rangle + \vert 11 \rangle)$

In [8]:
primitive = {'00': 1, '11': 1}
coeff = 1 / np.sqrt(2)
bell = StateFn(primitive, coeff)

for basis in ['00', '01', '10', '11']:
    print(f"'bell' maps |{basis}> to {bell.eval(basis)}")

'bell' maps |00> to 0.7071067811865475
'bell' maps |01> to 0.0
'bell' maps |10> to 0.0
'bell' maps |11> to 0.7071067811865475


As you know there are a lot of quantum states that can be described as tensor products of one-qubit states. The Bell state we have created above is not a such state, however e.g. the 2-qubit uniform superposition state $\frac{1}{2}(\vert 00 \rangle + \vert 01 \rangle + \vert 10 \rangle + \vert 11 \rangle) = \frac{1}{\sqrt{2}}(\vert 0 \rangle + \vert 1 \rangle) \otimes \frac{1}{\sqrt{2}}(\vert 0 \rangle + \vert 1 \rangle)$ is. Instead of specifying 2-qubit amplitudes to create a state function, as we did for the Bell state, we can use the tensoring capability of state functions to create the uniform superposition state. Tensoring of state functions can be done with the `.tensor()` function (`Plus.tensor(Plus)`) or the `^` symbol (`Plus ^ Plus`). Tensoring can be used also to do tensor powers, i.e. the second operand of tensoring can be an integer instead of a state function  and this will mean the tensor power of the first operand. E.g. let us create a uniform superposition state for five qubits

In [9]:
uniform = Plus ^ 5
uniform.primitive.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     └───┘

We have described the introductory basics of Aqua state functions above. Of course they support much more sophisticated functionality which is not covered in this notebook. A few honorable mentions are the functions `sample()` and `to_matrix()` which we encourage the readers to investigate on their own.

## Operators and Evolutions

### Operators
Now as we can build state functions we can define operations to transform them. Aqua operators do exactly this. They can be created using a few different primitives as well. Let us define an operator which takes the $\vert - \rangle$ state to the basis state $\vert 1\rangle$. As you know we could accomplish this with a Hadamard gate if we were building a quantum circuit. Let us use such a circuit as a primitive to create an operator

In [10]:
qc_hadamard = QuantumCircuit(1)
qc_hadamard.h(0)
op_hadamard = CircuitOp(qc_hadamard)

To make the operator `op_hadamard` act on the state `Minus` we need to chain them together. The official term for this in Aqua is _composition_. The symbol `@` can be used to compose the operator with the state function, or the `.compose()` function: they are equivalent

In [11]:
(op_hadamard @ Minus).eval().sample()

{'1': 1.0}

In [12]:
op_hadamard.compose(Minus).eval().sample()

{'1': 1.0}

If you check the type of the object `(op_hadamard @ Minus)` you will see that it is `CircuitStateFn`. The pre-built state function `Minus` is based on a circuit primitive and the composition of `op_hadamard` with it is straightforward to handle -  the result is a state function with the concatenated circuit as its primitive. In practice you will often come across more complicated compositions where the result is not so straightforward to obtain. In such cases an object of type `ComposedOp` will be created when you do a composition. The class `ComposedOp` is designed to represent compositions without doing the underlying calculations to obtain the actual result of the composition. This is called lazy representation and it is done to defer heavy numerical calculations to a later stage where posssibly a quantum device will be used and numerical simulation not needed. The function `.eval()` can be used to force the system to actually do the calculations and yield the final result. Lazy representation of various objects is ubiquitous in Aqua, `ComposedOp` is just one example of it. We will see some other examples later in this tutorial. 

Linear operators are linear maps between finite dimensional vector spaces, and therefore have a matrix representation. For e.g. the Pauli $X$ operator has the following matrix representation $X(\vert 0 \rangle, \vert 0 \rangle) = 0$, $X(\vert 1 \rangle, \vert 0 \rangle) = 1$, $X(\vert 0 \rangle, \vert 1 \rangle) = 1$, $X(\vert 1 \rangle, \vert 1 \rangle) = 0$. We can define a corresponding operator based on the matrix representation 

In [13]:
matrix_x = np.array([[0, 1], [1, 0]])
op_x = MatrixOp(matrix_x)

We can see the action of `op_x` by composing it with a state function as we did before for `op_hadamard`.

In [14]:
(op_x @ Zero).eval().sample()

{'1': 1.0}

In [15]:
(op_x @ One).eval().sample()

{'0': 1.0}

Pauli operators are an important class of operators that we routinely encounter in quantum computing. One particular reason for that is that the Pauli matrices (and their tensor products) form a matrix basis. Qiskit Terra provides implementations of Pauli gates, and we can define operators using Terra's `Pauli` module

In [16]:
op_x_terra = PauliOp(Pauli(z=[False], x=[True]))
print(repr(op_x_terra))

PauliOp(Pauli(z=[False], x=[True]), coeff=1.0)


In addition to the Pauli operators, there are a handful of other operators that are commonly used as building blocks for quantum applications. Aqua has a set of immutable `Operator` instances for this purpose. These are the Pauli Operators (`X`, `Y`, `Z`, `I`), Clifford Operators (`CX`, `S`, `H`, `Swap`, `CZ`), and the $T$ Operator (`T`). We can convert between the different operator representations by using `.to_circuit_op()`, `.to_matrix_op()`, and `to_pauli_op()` functions. These classes have additional methods too. Some very helpful ones are `add`, `mul`, `power`. We encourage the reader to explore these.

### Evolutions
The idea of evolution is central to quantum mechanics. The most common evolution that we encounter is the *time* evolution, generated by the Hamiltonian of the system. Operating on the state function with $e^{-iHt}$, where $H$ is the Hamiltonian, gives the time evolved form of the state function. More generally the operator for an evolution generated by $U$ is $e^{-iU\theta}$, where $\theta$ is the evolution parameter. Let us now see an example of how we can create an operator for evolution in Aqua.
We start with a simple two qubit Hamiltonian $H = XX + YY + ZZ$. This is a special case of the Heisenberg Hamiltonian, but for now we will call it the Heisenberg Hamiltonian.

In [17]:
hamiltonian = (X^X) + (Y^Y) + (Z^Z)
print(hamiltonian)

SummedOp([
  XX,
  YY,
  ZZ
])


There is a quite a bit to unpack here. First notice the use of `^`. In Python `^` is used as the bitwise XOR operator, however in Aqua, it has been overloaded to represent tensor products, as we have seen earlier for state functions. Note that you should always use parenthesis to indicate operator precedence, since built-in precedence of such operators in python will most likely not be what you are looking for (it is possible to overload the functionality of an operator, but not the precedence logic, so we have to stick with the built-in precedences). We also see a new class `SummedOp`, which is used to do lazy representation of operators formed by summing other operators. There are other classes to represent operators formed from other operators like `TensoredOp` (for tensor products), `ComposedOp` (composition), `ListOp` (list concatenation). We encourage you to explore these. Now we will see the evolution operator formed from this Hamiltonian.

In [18]:
evolution_param = Parameter('θ')
evolution_op = (evolution_param * hamiltonian).exp_i()
print(evolution_op)

e^(-i*1.0*θ * SummedOp([
  XX,
  YY,
  ZZ
]))


It might seem like we are just pretty printing an evolution operator, but convince ourselves that it is indeed what is happening under the hood, let us take a more detailed look at the evolution operator.

In [19]:
print(repr(evolution_op))

EvolvedOp(SummedOp([PauliOp(Pauli(z=[False, False], x=[True, True]), coeff=1.0), PauliOp(Pauli(z=[True, True], x=[True, True]), coeff=1.0), PauliOp(Pauli(z=[True, True], x=[False, False]), coeff=1.0)], coeff=1.0*θ, abelian=False), coeff=1.0)


We see a new class `EvolvedOp`. `EvolvedOp` is a placeholder for an evolution algorithm for later conversion into an Operator that approximates the exponentiation of the generator. Why a placeholder, and not an immediate conversion? This has all to do with the lazy execution of operators. Also notice that though the Hamiltonian is a sum of tensor products of Pauli operators, we do not see `TensoredOp` in the representation. This is because tensor products of Pauli operators are so ubiquitous, that the `PauliOp` class has been designed to handle those as well, in addition to the single qubit example that we saw earlier. 

When it comes to actually representing evolutions with operators instead of the lazy representation by `EvolvedOp`, there are  a few methods to construct the approximations, like `PauliTrotterEvolution`, and `MatrixEvolution`, to convert any `EvolvedOp` object into an actual sequence of operators. We will discuss them in greater detail in later tutorials. But in the meanwhile feel free to check them out.

## Measurements and Expectations

### Measurements

A Measurement is a functional that maps State functions to complex numbers. In Aqua, we think of a Measurement as the adjoint of a State function. For example say we want to measure $\langle 0 \vert 1 \rangle$. We can think of this as the functional $\vert 0 \rangle_{meas}()$, that accepts the State function $\vert 1 \rangle$, and maps it to a complex number, in this case $0$: $\vert 0 \rangle_{meas}(\vert 1 \rangle) = \langle 0 \vert 1 \rangle = 0$. Essentially, we are identifying $\vert 0 \rangle_{meas}()$ with $\langle 0 \vert$, the adjoint of $\vert 0 \rangle$. Thus $\langle 0 \vert$ is interpreted as a Measurement in the language of Aqua.

In [20]:
print(f"<0|: {Zero.adjoint()}")
print(f"<0|1>: {Zero.adjoint().eval(One)}")
print(f"<0|+>: {Zero.adjoint().eval(Plus)}")
print(f"<1|Z|+>: {(One.adjoint() @ Z @ Plus).eval()}")

<0|: DictMeasurement({'0': 1})
<0|1>: 0.0
<0|+>: (0.7071067811865476+0j)
<1|Z|+>: (-0.7071067811865476+8.7e-17j)


In the first section of this notebook we saw a bunch of methods to create state functions and think of them as ket state vectors at the back of our mind. What if we work with a system which does not have a state described in terms of ket state vectors? Such are the systems which have state representation in terms of density matrix (or density operator). Aqua has a class `OperatorStateFn` which allows the creation of state functions based on operators. Besides the fact that this class can be used to create state functions representing density operators, they have one important use case - measurements of observables. To measure the expectation value of some physical quantity (e.g. the hamiltonian) you need to build necessary measurement logic, which can be done just by creating an `OperatorStateFn` object by using the operator of the observable as its primitive, and take the adjoint of the result (i.e. make it a measurement as described above). We will see the usage of `OperatorStateFn` in this manner at the complete end-to-end example presented at the end of this notebook.

### Expectations

`ExpectationBase` and its subclasses enable the computation of expectation values of Observables. These are converters that replace `OperatorStateFn` measurements with equivalent measurements (or approximations thereof) that can be actually computed on a quantum or classical hardware. 

In [21]:
hamiltonian_meas = StateFn(hamiltonian).adjoint()
print(PauliExpectation().convert(hamiltonian_meas))

SummedOp([
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      ZZ
    ])),
         ┌───┐
    q_0: ┤ H ├
         ├───┤
    q_1: ┤ H ├
         └───┘
  ]),
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      ZZ
    ])),
         ┌─────┐┌───┐
    q_0: ┤ SDG ├┤ H ├
         ├─────┤├───┤
    q_1: ┤ SDG ├┤ H ├
         └─────┘└───┘
  ]),
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      ZZ
    ])),
    II
  ])
])


In the above example `PauliExpectation` is a subclass of `ExpectationBase` that converts Pauli-basis Observables to a diagonal $\{Z, I\}^{\otimes n}$ basis (computational basis) by appending circuit post-rotations to the measured State function. In the Heisenberg Hamiltonian only $ZZ$ is diagonal. To measure $X$ ($Y$) in the computational basis, we have to rotate the measurement by $H$ ($S^\dagger H$). This is exactly what the `PauliExpectation` class did. Let us look at another example to see another important feature of the `PauliExpectation` class.

In [22]:
hamiltonian2 = (I^X) + (X^I)
hamiltonian2_meas = StateFn(hamiltonian2).adjoint()
print(PauliExpectation().convert(hamiltonian2_meas))

ComposedOp([
  OperatorMeasurement(AbelianSummedOp([
    IZ,
    ZI
  ])),
       ┌───┐
  q_0: ┤ H ├
       ├───┤
  q_1: ┤ H ├
       └───┘
])


In the above example, the two Pauli strings $IX$, and $XI$ commute, and can therefore be measured simultaneously. This reduces the number of circuits required to measure the Hamiltonian. The `PauliExpectation` class by default groups commuting Pauli strings into an `AbelianSummedOp` to facilitate the simultaneous measurement. This feature can be turned off by setting `group_paulis` to `False`, when instantiating a `PauliExpectation` class. The other `ExpectationBase` subclasses are  `AerPauliExpectation`, `MatrixExpectation`, and `CVaRExpectation`. There is also a `ExpectationFactory` for conveniently selecting an `ExpectationBase` subclass. We encourage you to check them out.

## Example

Now that we have seen the building blocks of Aqua's Operator flow, let us see a complete example to get a feel of what we can do with it. We will attempt a Trotter-Suzuki expansion of the Heisenberg Hamiltonian, and calculate the associated energies.

In [23]:
# Heisenberg Hamiltonian
hamiltonian = (X^X) + (Y^Y) + (Z^Z)

# Initial State -- Bell state Phi+
bell = CX @ (Plus^Zero)

# Time evolution operator
evolution_param = Parameter("θ")
evolution_op = (evolution_param * hamiltonian).exp_i()

# Measurement of the Hamiltonian with the time evolved state
evo_and_meas = StateFn(hamiltonian).adjoint() @ evolution_op @ bell

# Trotterization
trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki()).convert(evo_and_meas)

# Diagonalize the measurement
diagonalized_meas_op = PauliExpectation().convert(trotterized_op)

# Measure energies by Trotterization an sampling
time_array = list(range(10))
expectations = diagonalized_meas_op.bind_parameters({evolution_param: time_array})
sampler = CircuitSampler(backend=BasicAer.get_backend("qasm_simulator"))
sampled_expectations = sampler.convert(expectations)
sampled_energies = sampled_expectations.eval()
print(f"Sampled energies after Trotterization:\n {np.real(sampled_energies)}")

Sampled energies after Trotterization:
 [ 0.00195313 -0.07226562  0.0703125  -0.00585937  0.00195312  0.02539062
 -0.04492188  0.05273438  0.04296875  0.05664062]


In [24]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright